In [1]:
import pandas as pd
import numpy as np
import math
from pyquaternion import Quaternion
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

data=pd.read_excel('HW5-2.xls')
given=data.to_numpy()
Data=np.zeros([100,4])
%matplotlib tk

iter=2000
lr=0.001
q=np.array([1,0,0,0])
q_predict=np.zeros([100,4])

In [2]:
ax=data.loc[:,['ax']]
ay=data.loc[:,['ay']]
az=data.loc[:,['az']]
ax1=np.array(ax)
ay1=np.array(ay)
az1=np.array(az)

In [3]:
for i in range(100):
    for j in range(iter):
        #normalize後的acc只有方向沒有大小
        norm=math.sqrt(ax1[i,0]*ax1[i,0]+ay1[i,0]*ay1[i,0]+az1[i,0]*az1[i,0])
        a_x=ax1[i,0]/norm
        a_y=ay1[i,0]/norm
        a_z=az1[i,0]/norm
        #cost function=e
        f1=-2*(q[1]*q[3]-q[2]*q[0])-a_x
        f2=-2*(q[2]*q[3]+q[1]*q[0])-a_y
        f3=-1+2*(q[1]*q[1]+q[2]*q[2])-a_z
        f=np.array([f1,f2,f3])
        #Jacobian
        J=np.array([[2*q[2],-2*q[3],2*q[0],-2*q[1]],[-2*q[1],-2*q[0],-2*q[3],-2*q[2]],[0,4*q[1],4*q[2],0]])
        #Gradient
        G=(J.T).dot(f)
        #normalize Gradient
        norm_G=math.sqrt(G[0]*G[0]+G[1]*G[1]+G[2]*G[2]+G[3]*G[3])
        G=G/norm_G
        #Quaternion
        q=q-lr*G
        # normalize quaternion
        norm_q=math.sqrt(q[0]*q[0]+q[1]*q[1]+q[2]*q[2]+q[3]*q[3])
        q=q/norm_q
        
    q_predict[i,0]=q[0]
    q_predict[i,1]=q[1]
    q_predict[i,2]=q[2]
    q_predict[i,3]=q[3]
    
    
    
    q_1=Quaternion(q[0],q[1],q[2],q[3])
    q_1_inverse=q_1.inverse
    E=Quaternion(0,0,0,-9.8)
    #gravity to sensor frame
    q_2=q_1_inverse*E*q_1
    
    for s in range(4):
        Data[i,s]=q_2[s]




In [4]:
q

array([ 0.67005393,  0.73159257, -0.0439033 ,  0.11778172])

In [6]:
Data = Data[:,1:]
Given_draw=np.zeros([100,6])
Data_draw=np.zeros([100,6])
Data_draw[:,3:]=Data
Given_draw[:,3:]=given

soa1=Data_draw
soa2=Given_draw

X1,Y1,Z1,U1,V1,W1=zip(*soa1)
X2,Y2,Z2,U2,V2,W2=zip(*soa2)
fig=plt.figure(figsize=plt.figaspect(0.5))
ax=fig.add_subplot(1,1,1,projection='3d')

l1=ax.quiver(X1,Y1,Z1,U1,V1,W1,color='r',pivot='tail',length=1)
l2=ax.quiver(X2,Y2,Z2,U2,V2,W2,color='b',pivot='tail',length=1)

ax.quiver(0,0,0,3,0,0,color='gray',pivot='tail',length=1)
ax.quiver(0,0,0,0,3,0,color='gray',pivot='tail',length=1)
ax.quiver(0,0,0,0,0,3,color='gray',pivot='tail',length=1)

ax.set_xlim([-9.8,5])
ax.set_ylim([-9.8,5])
ax.set_zlim([-9.8,5])

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z(gravity=-9.8)')
plt.title("Attitude")
plt.legend(handles=[l2,l1],labels=['Given','Pre'],loc='best')

plt.show()

ValueError: could not broadcast input array from shape (100,2) into shape (100,3)

In [6]:
df1=pd.DataFrame(Data)
filepath='Acc_pre.xlsx'
df1.to_excel(filepath,index=False)

df2=pd.DataFrame(q_predict)
filepath='Quaternion_pre.xlsx'
df2.to_excel(filepath,index=False)

NameError: name 'soa1' is not defined